## Name Matching



In [105]:
#  Importing libraries and module and some setting for notebook

import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct  #Cosine Similarity
import time
import sys
pd.set_option('display.max_colwidth', -1)

dataset_path = '/Users/friday/Library/CloudStorage/OneDrive-Personal/Documents/MS/UMCP/RA/Assignment-1_Matching-_two_dbs/Datasets/Firm_classification'
# reading dataset as df
# path = '/Users/friday/Library/CloudStorage/OneDrive-Personal/Documents/MS/UMCP/RA/Assignment-1_Matching-_two_dbs/Datasets/company_names_db1.csv'
compustat_file_name = 'compustat_deepak.xlsx'
fda_file = 'Firm classifiers.xlsx'
df_compustat =  pd.read_excel(dataset_path + "/" + compustat_file_name)
df_fda_firms =  pd.read_excel(dataset_path + "/" + fda_file)

/var/folders/tn/k31185892wl6cj_lpfjjr2wc0000gn/T/ipykernel_19556/4025729238.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [91]:
df_compustat.shape

(9456, 15)

In [113]:
df_compustat.head()

,Global Company Key,Ticker Symbol,CUSIP,Company Name,FDA DB,FEI Number,Project Area,Product Type,Product code,NAICS code,NAICS description,FSMA firms,FDA but not FSMA,CPSC firms,NHTSA firms
0,1004,AIR,000361105,aar corp,NaN,NaN,NaN,NaN,NaN,423860.0,Transportation Equipment and Supplies (except Motor Vehicle) Merchant Wholesalers,NaN,NaN,NaN,NaN
1,1045,AAL,02376R102,american airlines group inc,NaN,NaN,NaN,NaN,NaN,481111.0,Scheduled Passenger Air Transportation,NaN,NaN,NaN,NaN
2,1050,CECO,125141101,ceco environmental corp,NaN,NaN,NaN,NaN,NaN,333413.0,Industrial and Commercial Fan and Blower and Air Purification Equipment Manufacturing,NaN,NaN,NaN,NaN
3,1062,ASA,G3156P103,asa gold and precious metals,NaN,NaN,NaN,NaN,NaN,523999.0,Miscellaneous Financial Investment Activities,NaN,NaN,NaN,NaN
4,1075,PNW,723484101,pinnacle west capital corp,NaN,NaN,NaN,NaN,NaN,2211.0,"Electric Power Generation, Transmission and DistributionT",NaN,NaN,NaN,NaN


In [112]:
df_fda_firms.head()

,FEI Number,Legal Name,Project Area,Product Type,Product code
0,3008839110,"pollen collection & sales, inc.",Foodborne Biological Hazards,Food/Cosmetics,NaN
1,3013285591,patagonia organic llc,Foodborne Biological Hazards,Food/Cosmetics,NaN
2,3012645346,bar giacosa corp dba bar pitti,Foodborne Biological Hazards,Food/Cosmetics,NaN
3,3011462240,blood bank of san bernardino and riverside counties,Blood and Blood Products,Biologics,NaN
4,3012233744,sigona's fruit for thought,"Food Composition, Standards, Labeling and Econ",Food/Cosmetics,NaN


In [108]:
compustat_firm_names = df_compustat['Company Name'].str.lower()
compustat_firm_names = pd.DataFrame({'Company Name':compustat_firm_names.values})

In [109]:
compustat_firm_names.head()

,Company Name
0,aar corp
1,american airlines group inc
2,ceco environmental corp
3,asa gold and precious metals
4,pinnacle west capital corp


In [110]:
df_fda_firms['Legal Name'] = df_fda_firms['Legal Name'].str.lower() 

In [111]:
df_compustat['Company Name'] = df_compustat['Company Name'].str.lower()

In [5]:
df_compustat.shape

(9456, 15)

In [6]:
compustat_firm_names.shape

NameError: name 'compustat_firm_names' is not defined

In [95]:
#path3 = '/Users/friday/Library/CloudStorage/OneDrive-Personal/Documents/MS/UMCP/RA/Assignment-1_Matching-_two_dbs/Datasets/compustat_w_naics (3).xlsx'

In [55]:
#compustate_df =  pd.read_excel(path3)

In [4]:
#compustate_df.head()

In [57]:
#df_all_firms = compustate_df[['Company Name', 'Ticker Symbol', 'North American Industry Classification Code']]

In [5]:
#compustate_df.head()

In [6]:
#df2.shape

In [7]:
#df1.head()

In [98]:
#df1.shape

In [51]:
#df_temp = df.copy(deep = False)

In [62]:
#df_temp.shape

(119328, 2)

In [10]:
df_all_firms = pd.concat([df_compustat['Company Name'], df_fda_firms['Legal Name']], axis=0)
df_all_firms = pd.DataFrame({'US Company Name':df_all_firms.values})
#df_all_firms = df_all_firms['US Company Name'].str.lower()
#df_all_firms = pd.DataFrame({'US Company Name':df_all_firms.values})

In [11]:
df_all_firms.head()

,US Company Name
0,aar corp
1,american airlines group inc
2,ceco environmental corp
3,asa gold and precious metals
4,pinnacle west capital corp


In [12]:
df_all_firms.shape

(287187, 1)

In [11]:
#df = pd.DataFrame({'No':df.index, 'US Company Name':df.values})

In [12]:
#df.head()

## ngrams 

In [13]:
#  ngrams(here we are taking n = 3 thus 3-gram (trigrams )
#  used for cleaning and removing some punctuation (dots, comma’s etc) i.e.((,-./)) from a string 
#  and generate and collect all n-grams of the string.  

 
def ngrams(string, n=4):

    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

# Testing ngrams work for verification 

print('All 3-grams in "EXPEDITORS INTERNATIONAL":')
ngrams('EXPEDITORS INTERNATIONAL')

All 3-grams in "EXPEDITORS INTERNATIONAL":


['EXPE',
 'XPED',
 'PEDI',
 'EDIT',
 'DITO',
 'ITOR',
 'TORS',
 'ORS ',
 'RS I',
 'S IN',
 ' INT',
 'INTE',
 'NTER',
 'TERN',
 'ERNA',
 'RNAT',
 'NATI',
 'ATIO',
 'TION',
 'IONA',
 'ONAL']

## TF-IDF and Vectorization

In [14]:
# After having each words split (token or  lemmas (n-gram generated items) ) into a vector and
# Scikit-learn’s  Tfidfvectorizer aim to do the same thing, which is to convert a collection of raw documents to a matrix of TF-IDF features. 
# Generate the matrix of TF-IDF (Term Frequency-Inverse Document frequency)values for each

firms = df_all_firms['US Company Name']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(firms.values.astype('U'))

In [15]:
# View sparse CSR matrix.
print(tf_idf_matrix[0])


  (0, 40052)	0.2954284285021588
  (0, 3831)	0.29446570389039534
  (0, 105640)	0.4005237730012691
  (0, 30500)	0.49057354365493394
  (0, 25824)	0.6518667796350318


## Optimized Cosine Similarity  by  sparse_dot_topn 

In [16]:
# calculate the similarity between two vectors of TF-IDF values the Cosine Similarity is usually used.
# result matrix in a very sparse terms and Scikit-learn deals with this nicely by returning a sparse CSR matrix.

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))


In [17]:
#  Run the optimized cosine similarity function. 
#  Only stores the top 10 most similar items with a similarity above 0.8

t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 76.10653805732727


In [18]:
# unpacks the resulting sparse matrix

def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [16]:
firms.shape

(287187,)

In [19]:
# store the  matches into new dataframe called matched_df and printing 10 samples

matches_df = get_matches_df(matches, firms, top=200)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # For removing all exact matches
matches_df.sort_values('similairity', ascending=False).head(100)

,left_side,right_side,similairity
161,barrick gold corp,barrick gold corpo,0.982770
172,becton dickinson & co,"becton dickinson & co., ltd.",0.972535
117,archer-daniels-midland co,archer-daniels-midland company,0.964211
187,bio-rad laboratories inc,bio-rad laboratories,0.956718
173,becton dickinson & co,becton dickinson & company,0.953255
53,honeywell international inc,honeywell international,0.952129
15,abbott laboratories,"abbott laboratories, inc.",0.951230
16,abbott laboratories,"abbott laboratories, inc",0.951230
17,abbott laboratories,abbott laboratories co.,0.944431
125,arrow electronics inc,arrow electronics,0.944209


In [110]:
#matches_df = get_matches_df(matches, room_types)

In [20]:
matches_df = get_matches_df(matches, firms, top=firms.shape[0])

In [21]:
matches_df.shape

(287187, 3)

In [22]:
# printing the matched in sorted order 

matches_df.sort_values(['similairity'], ascending=False).head(10)

,left_side,right_side,similairity
75110,geoje fisheries cooperatives marine product processing plant,geoje fisheries cooperatives marine product processing plant,1.0
79083,billings clinic reproductive medicine and fertility care,billings clinic reproductive medicine and fertility care,1.0
29208,"quality builders, llc dba blue valley aquaculture","quality builders, llc dba blue valley aquaculture",1.0
67925,sonoma specialty food corporation,sonoma specialty food corporation,1.0
50503,"key distributed products dba advanced vision technologies, inc.","key distributed products dba advanced vision technologies, inc.",1.0
111079,"terminal freezers, llc a lineage logistics company","terminal freezers, llc a lineage logistics company",1.0
287000,productores dulces y galletas cabadas de la piedad sa cv,productores dulces y galletas cabadas de la piedad sa cv,1.0
247826,valeant laboratorios grossman sa,valeant laboratorios grossman sa,1.0
85191,transcendencias comerciales s.l.,transcendencias comerciales s.l.,1.0
240237,aogrand international trade corporation,aogrand international trade corporation,1.0


In [114]:
#matches_df.to_csv('matches_both_files.csv')

In [23]:
def is_similar(left, right, score):
    left = str(left).replace("-", "")
    right = str(right).replace("-","")
    left = str(left).replace("THE", "")
    right = str(right).replace("THE","")
    left = str(left).replace("The", "")
    right = str(right).replace("The","")
    left = str(left).split(" ")
    right = str(right).split(" ")
    if (len(left[0])) >=3:
        if left[0][0:3] == right[0][0:3]:
            return True
        else:
            return False
    else:
        return True
    '''
    if score < 0.8:
        return False
    elif len(left) != len(right):
        return True
    
    for word in left.copy():
        if word in right:
            left.pop(left.index(word))
            right.pop(right.index(word))
    if len(left) == len(right) and len(left) == 0:
        return True
    max_check = 1
    if len(left[0]) >= 3:
        max_check = 3
    else:
        max_check = len(left[0])
    if left[0][0:max_check] == right[0][0:max_check]:
        return True
    else:
        return False
    '''

In [22]:
is_similar('1800FLOWERS', '1-800-FLOWERS.COM', 0.7)

True

In [24]:
t1 = time.time()
matches_df['is_similar'] = matches_df.apply(lambda row : is_similar(row.left_side, row.right_side, row.similairity), axis = 1)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 4.571101903915405


In [118]:
#matches_df[matches_df['is_compustat'] == 'False'].head()

In [27]:
matches_df.head()

,left_side,right_side,similairity,is_similar,is_compustat
0,aar corp,aar corp,1.000000,True,True
1,american airlines group inc,"american airlines group, inc",1.000000,True,True
2,american airlines group inc,american airlines group inc,1.000000,True,True
3,american airlines group inc,american airlines group,0.928566,True,True
4,american airlines group inc,"american airlines, inc",0.801991,True,True


In [123]:
matches_df.to_csv('matches_fda_compustat.csv')

In [61]:
'american airlines group inc' in compustat_firm_names['Company Name'].tolist()

True

In [29]:
'ABBOTT LABORATORY' in df_fda_firms['Legal Name'].tolist()

True

## writing new logic to filter out the firm names

In [81]:
#matches_df['is_import_export'] = matches_df.apply(lambda row : row.left_side in df1['US Company Name'].tolist(), axis = 1 )

In [26]:
t1 = time.time()
matches_df['is_compustat'] = matches_df.apply(lambda row : row.left_side in df_compustat['Company Name'].tolist(), axis = 1 )
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 45.57412123680115


In [66]:
t1 = time.time()
matches_df['is_fda_left'] = matches_df.apply(lambda row : row.left_side in df_fda_firms['Legal Name'].tolist(), axis = 1 )
t = time.time()-t1
print("SELFTIMED:", t)

KeyboardInterrupt: 

In [ ]:
t1 = time.time()
matches_df['is_fda_right'] = matches_df.apply(lambda row : row.right_side in df_fda_firms['Legal Name'].tolist(), axis = 1 )
t = time.time()-t1
print("SELFTIMED:", t)

In [28]:
t1 = time.time()
matches_df['different_database'] = matches_df.apply(lambda row : row.left_side in df_compustat['Company Name'].tolist() and row.right_side in df_fda_firms['Legal Name'].tolist(), axis = 1)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 112.46582007408142


In [55]:
actual_match = matches_df.drop_duplicates(subset='left_side', keep="first")

(287187, 6)

In [31]:
matches_df[(matches_df.is_compustat == True) & (matches_df.different_database == True)].shape

(2617, 6)

In [102]:
matches_df[matches_df.left_side == 'eisai co ltd'].drop_duplicates()

,left_side,right_side,similairity,is_similar,is_compustat,different_database
7585,eisai co ltd,eisai co ltd,1.0,True,True,True
7586,eisai co ltd,"eisai co., ltd.",1.0,True,True,True


In [32]:
actual_match = matches_df[(matches_df.is_compustat == True) & (matches_df.different_database == True)].drop_duplicates(subset='left_side', keep="first")

In [103]:
actual_match1 = matches_df[(matches_df.is_compustat == True) & (matches_df.different_database == True) & (matches_df.is_similar == True)].drop_duplicates(subset='left_side', keep="first")

In [104]:
actual_match1.shape

(864, 6)

In [40]:
#matches_df[matches_df.different_database == True].shape

In [34]:
#actual_temp = matches_df.loc[(matches_df.is_compustat == True)].drop_duplicates(subset='right_side', keep="first")

In [39]:
#actual_temp.loc[(matches_df.different_database == True)].shape

In [38]:
#actual_temp.head()

In [137]:
#actual_matches = matches_df.loc[(matches_df.is_compustat == True) & (matches_df.different_database == True)]

In [139]:
#actual_matches.shape

(523, 6)

In [140]:
#actual_matches.drop_duplicates(inplace = True)

/var/folders/tn/k31185892wl6cj_lpfjjr2wc0000gn/T/ipykernel_17072/3249037189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_matches.drop_duplicates(inplace = True)


In [41]:
#actual_match.shape

In [42]:
#temp = matches_df.loc[(matches_df.is_compustat == True)]

In [43]:
#temp.drop_duplicates(inplace = True)

In [44]:
#temp.shape

In [49]:
#matches_df.drop('is_import_export', axis=1, inplace=True)

In [45]:
#matches_df.shape

In [46]:
#matches_df.drop_duplicates(inplace = True)

## New code

In [94]:
actual_match

,left_side,right_side,similairity,is_similar,is_compustat,different_database
1,american airlines group inc,"american airlines group, inc",1.000000,True,True,True
13,abbott laboratories,abbott laboratories,1.000000,True,True,True
26,acme united corp,acme united corporation,0.899275,True,True,True
33,air products & chemicals inc,"air products & chemicals, inc",1.000000,True,True,True
47,petro usa inc,petro usa,0.876356,True,True,True
...,...,...,...,...,...,...
11347,banco santander brasil -adr,banco santander brasil,0.841037,True,True,True
11350,valneva se,valneva,0.836074,True,True,True
11353,kamada ltd,kamada ltd.,1.000000,True,True,True
11371,cnh industrial nv,cnh industrial,0.839784,True,True,True


In [96]:
actual_match[actual_match.is_similar == True and actual_match.right_side].shape

(863, 6)

In [99]:
actual_match[actual_match.left_side == "tilly's inc"]

,left_side,right_side,similairity,is_similar,is_compustat,different_database
10956,tilly's inc,willy's inc,0.860486,False,True,True


In [67]:
actual_match[actual_match.left_side  == 'petro usa inc'].right_side.values[0]

'petro usa'

In [69]:
df_compustat.shape

(9456, 15)

In [115]:
df_compustat.head()

,Global Company Key,Ticker Symbol,CUSIP,Company Name,FDA DB,FEI Number,Project Area,Product Type,Product code,NAICS code,NAICS description,FSMA firms,FDA but not FSMA,CPSC firms,NHTSA firms
0,1004,AIR,000361105,aar corp,NA,NaN,NaN,NaN,NaN,423860.0,Transportation Equipment and Supplies (except Motor Vehicle) Merchant Wholesalers,NaN,NaN,NaN,NaN
1,1045,AAL,02376R102,american airlines group inc,"american airlines group, inc",NaN,NaN,NaN,NaN,481111.0,Scheduled Passenger Air Transportation,NaN,NaN,NaN,NaN
2,1050,CECO,125141101,ceco environmental corp,NA,NaN,NaN,NaN,NaN,333413.0,Industrial and Commercial Fan and Blower and Air Purification Equipment Manufacturing,NaN,NaN,NaN,NaN
3,1062,ASA,G3156P103,asa gold and precious metals,NA,NaN,NaN,NaN,NaN,523999.0,Miscellaneous Financial Investment Activities,NaN,NaN,NaN,NaN
4,1075,PNW,723484101,pinnacle west capital corp,NA,NaN,NaN,NaN,NaN,2211.0,"Electric Power Generation, Transmission and DistributionT",NaN,NaN,NaN,NaN


In [114]:
t1 = time.time()
df_compustat['FDA DB'] = df_compustat.apply(lambda row : actual_match1[actual_match1.left_side  == row['Company Name']].right_side.values[0] if row['Company Name'] in actual_match1.left_side.tolist() else 'NA', axis = 1)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.4195568561553955


In [75]:
df_fda_firms[df_fda_firms['Legal Name'] == 'american airlines group, inc']['FEI Number'].values[0]

1721149

In [116]:
t1 = time.time()
df_compustat['FEI Number'] = df_compustat.apply(lambda row : df_fda_firms[df_fda_firms['Legal Name'] == row['FDA DB']]['FEI Number'].values[0] if row['FDA DB'] in df_fda_firms['Legal Name'].tolist() else 'NA', axis = 1)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 75.74697589874268


In [117]:
t1 = time.time()
df_compustat['Project Area'] = df_compustat.apply(lambda row : df_fda_firms[df_fda_firms['Legal Name'] == row['FDA DB']]['Project Area'].values[0] if row['FDA DB'] in df_fda_firms['Legal Name'].tolist() else 'NA', axis = 1)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 75.69663381576538


In [120]:
t1 = time.time()
df_compustat['Product Type'] = df_compustat.apply(lambda row : df_fda_firms[df_fda_firms['Legal Name'] == row['FDA DB']]['Product Type'].values[0] if row['FDA DB'] in df_fda_firms['Legal Name'].tolist() else 'NA', axis = 1)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 76.22486782073975


In [118]:
t1 = time.time()
df_compustat['Product code'] = df_compustat.apply(lambda row : df_fda_firms[df_fda_firms['Legal Name'] == row['FDA DB']]['Product code'].values[0] if row['FDA DB'] in df_fda_firms['Legal Name'].tolist() else 'NA', axis = 1)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 75.71704697608948


In [121]:
df_compustat.head()

,Global Company Key,Ticker Symbol,CUSIP,Company Name,FDA DB,FEI Number,Project Area,Product Type,Product code,NAICS code,NAICS description,FSMA firms,FDA but not FSMA,CPSC firms,NHTSA firms
0,1004,AIR,000361105,aar corp,NA,NA,NA,NA,NA,423860.0,Transportation Equipment and Supplies (except Motor Vehicle) Merchant Wholesalers,NaN,NaN,NaN,NaN
1,1045,AAL,02376R102,american airlines group inc,"american airlines group, inc",1721149,Technical Assistance: Food and Cosmetics,Food/Cosmetics,NaN,481111.0,Scheduled Passenger Air Transportation,NaN,NaN,NaN,NaN
2,1050,CECO,125141101,ceco environmental corp,NA,NA,NA,NA,NA,333413.0,Industrial and Commercial Fan and Blower and Air Purification Equipment Manufacturing,NaN,NaN,NaN,NaN
3,1062,ASA,G3156P103,asa gold and precious metals,NA,NA,NA,NA,NA,523999.0,Miscellaneous Financial Investment Activities,NaN,NaN,NaN,NaN
4,1075,PNW,723484101,pinnacle west capital corp,NA,NA,NA,NA,NA,2211.0,"Electric Power Generation, Transmission and DistributionT",NaN,NaN,NaN,NaN


In [122]:
df_compustat.to_excel('compustat_fda_matches1.xlsx')

In [87]:
df_compustat[df_compustat['Company Name'] == 'petro usa inc']

,Global Company Key,Ticker Symbol,CUSIP,Company Name,FDA DB,FEI Number,Project Area,Product Type,Product code,NAICS code,NAICS description,FSMA firms,FDA but not FSMA,CPSC firms,NHTSA firms
26,1272,PBAJ,71648P103,petro usa inc,petro usa,3010889567,Tobacco,Tobacco,NaN,447110.0,Gasoline Stations with Convenience Stores,NaN,NaN,NaN,NaN


In [92]:
df_compustat[df_compustat['Company Name'] == 'eisai co ltd']

,Global Company Key,Ticker Symbol,CUSIP,Company Name,FDA DB,FEI Number,Project Area,Product Type,Product code,NAICS code,NAICS description,FSMA firms,FDA but not FSMA,CPSC firms,NHTSA firms
6080,100418,ESALY,282579309,eisai co ltd,eisai co ltd,1000608517,NaN,NaN,"16YGH07 \ FISH MAW, FISHERY PRODUCTS, N.E.C.",325412.0,Pharmaceutical Preparation Manufacturing,NaN,1.0,NaN,NaN


In [63]:
df_compustat.head(10)

,Global Company Key,Ticker Symbol,CUSIP,Company Name,FDA DB,FEI Number,Project Area,Product Type,Product code,NAICS code,NAICS description,FSMA firms,FDA but not FSMA,CPSC firms,NHTSA firms
0,1004,AIR,000361105,AAR CORP,AAR CORP,NaN,NaN,NaN,NaN,423860.0,Transportation Equipment and Supplies (except Motor Vehicle) Merchant Wholesalers,NaN,NaN,NaN,NaN
1,1045,AAL,02376R102,AMERICAN AIRLINES GROUP INC,AMERICAN AIRLINES GROUP INC,NaN,NaN,NaN,NaN,481111.0,Scheduled Passenger Air Transportation,NaN,NaN,NaN,NaN
2,1050,CECO,125141101,CECO ENVIRONMENTAL CORP,CECO ENVIRONMENTAL CORP,NaN,NaN,NaN,NaN,333413.0,Industrial and Commercial Fan and Blower and Air Purification Equipment Manufacturing,NaN,NaN,NaN,NaN
3,1062,ASA,G3156P103,ASA GOLD AND PRECIOUS METALS,ASA GOLD AND PRECIOUS METALS,NaN,NaN,NaN,NaN,523999.0,Miscellaneous Financial Investment Activities,NaN,NaN,NaN,NaN
4,1075,PNW,723484101,PINNACLE WEST CAPITAL CORP,PINNACLE WEST CAPITAL CORP,NaN,NaN,NaN,NaN,2211.0,"Electric Power Generation, Transmission and DistributionT",NaN,NaN,NaN,NaN
5,1076,PRG,74319R101,PROG HOLDINGS INC,PROG HOLDINGS INC,NaN,NaN,NaN,NaN,522220.0,Sales Financing,NaN,NaN,NaN,NaN
6,1078,ABT,002824100,ABBOTT LABORATORIES,ABBOTT LABORATORIES,NaN,NaN,NaN,NaN,334510.0,Electromedical and Electrotherapeutic Apparatus Manufacturing,NaN,1.0,NaN,NaN
7,1084,WDDD,98159J200,WORLDS INC,WORLDS INC,NaN,NaN,NaN,NaN,519130.0,Internet Publishing and Broadcasting and Web Search Portals,NaN,NaN,NaN,NaN
8,1097,ACMTA,004616207,ACMAT CORP -CL A,ACMAT CORP -CL A,NaN,NaN,NaN,NaN,524126.0,Direct Property and Casualty Insurance Carriers,NaN,NaN,NaN,NaN
9,1104,ACU,004816104,ACME UNITED CORP,ACME UNITED CORP,NaN,NaN,NaN,NaN,332215.0,"Metal Kitchen Cookware, Utensil, Cutlery, and Flatware (except Precious) Manufacturing",NaN,NaN,1.0,NaN


In [89]:
matches_df.to_csv('matches13.csv')

In [59]:
'CATERPILLAR INC' in df1['US Company Name'].tolist()

False

In [63]:
# checking the firms which are not matching with the second database

In [66]:
df_unmatch_firms = df1[df1.apply(lambda row : row['US Company Name'] not in matches_df.left_side.tolist(), axis = 1)]

In [67]:
df_unmatch_firms.shape

(1541, 6)

In [68]:
df_unmatch_firms.to_csv('unmatched_firms.csv')

In [105]:
compustate_df[compustate_df['Company Name']  == 'AAR CORP']['Ticker Symbol'].values[0]

'AIR'

In [ ]:
# 371 firms present on compustat
# 1571 firms not matching
# 7492

In [98]:
matches_df.head()

,left_side,right_side,similairity,is_similar,different_database,is_compustat,is_import_export
0,OVERSEAS FORWARDING,OVERSEAS FORWARDING,1.0,True,False,False,True
1,BOSTON HIDES AND FURS,BOSTON HIDES AND FURS,1.0,True,False,False,True
2,EXPEDITORS INTERNATIONAL,EXPEDITORS INTERNATIONAL,1.0,True,False,False,True
3,SEA SHIPPING LINE,SEA SHIPPING LINE,1.0,True,False,False,True
4,SHIPCO TRANSPORT,SHIPCO TRANSPORT,1.0,True,False,False,True


In [106]:
matches_df['Ticker'] = matches_df.apply(lambda row : compustate_df[compustate_df['Company Name']  == row.right_side]['Ticker Symbol'].values[0] if row.right_side in compustate_df['Company Name'].tolist() else 'FALSE', axis = 1)

In [108]:
matches_df['NAICC'] = matches_df.apply(lambda row : compustate_df[compustate_df['Company Name']  == row.right_side]['North American Industry Classification Code'].values[0] if row.right_side in compustate_df['Company Name'].tolist() else 'FALSE', axis = 1)

In [109]:
matches_df[matches_df.different_database == True].head()

,left_side,right_side,similairity,is_similar,different_database,is_compustat,is_import_export,Ticker,NAICC
57,CATERPILLAR,CATERPILLAR INC,0.876564,True,True,True,True,CAT,333120.0
86,CH ROBINSON WORLDWIDE,C H ROBINSON WORLDWIDE INC,0.802611,True,True,True,True,CHRW,484121.0
94,WHIRLPOOL,WHIRLPOOL CORP,0.846576,True,True,True,True,WHR,3352.0
106,HANESBRANDS,HANESBRANDS INC,0.892917,True,True,True,True,HBI,315.0
141,FORD MOTOR,FORD MOTOR CO,0.845800,True,True,True,True,F,33611.0


In [113]:
results_df = pd.read_excel('/Users/friday/Library/CloudStorage/OneDrive-Personal/Documents/MS/UMCP/RA/Assignment-1_Matching-_two_dbs/Datasets/results_from_code.xlsx')
results_df.head()

,Company Name,ID
0,CARDINAL HEALTH,775
1,PRICESMART,3778
2,AKZO NOBEL,129
3,3M,7
4,WOLVERINE WORLD WIDE,5224


In [114]:
compustat_id_df = pd.read_excel('/Users/friday/Library/CloudStorage/OneDrive-Personal/Documents/MS/UMCP/RA/Assignment-1_Matching-_two_dbs/Datasets/firmsWithID.xlsx')
compustat_id_df.head()

,Compnay Name,ID
0,1-800-FLOWERS.COM,0
1,180 DEGREE CPTL CORP,1
2,1PM INDUSTRIES INC,2
3,1ST SOURCE CORP,3
4,360 SECURITY TECHNOLOGY INC,4


In [127]:
compustat_id_df[compustat_id_df['ID'] == 775]['Compnay Name'].values[0]

'CARDINAL HEALTH INC'

In [119]:
compustat_id_df[compustat_id_df[['ID'] == 775]]

KeyError: False

In [125]:
results_df['Compustat Company Name'] = results_df.apply(lambda row : compustat_id_df[compustat_id_df['ID'] == row['ID']]['Compnay Name'].values[0] if row['ID'] is not None else False, axis = 1)

In [129]:
results_df.head(100)

,Company Name,ID,Compustat Company Name
0,CARDINAL HEALTH,775,CARDINAL HEALTH INC
1,PRICESMART,3778,PRICESMART INC
2,AKZO NOBEL,129,AKZO NOBEL NV
3,3M,7,3M CO
4,WOLVERINE WORLD WIDE,5224,WOLVERINE WORLD WIDE
5,CONAGRA,1071,CONAGRA BRANDS INC
6,EASTMAN KODAK,1388,EASTMAN KODAK CO
7,CLEARWATER PAPER,978,CLEARWATER PAPER CORP
8,CATERPILLAR,802,CATERPILLAR FINANCIAL SVCS
9,JOHNSON CONTROLS,2722,JOHNSON CONTROLS INTL PLC


In [130]:
results_df.to_csv('results_from_code_with_names.csv')